<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Общее-впечатление-(ревью-2)" data-toc-modified-id="Общее-впечатление-(ревью-2)-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span><font color="orange">Общее впечатление (ревью 2)</font></a></span></li></ul></li><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#BERT-модель" data-toc-modified-id="BERT-модель-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>BERT модель</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Тест" data-toc-modified-id="Тест-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Тест</a></span></li></ul></li><li><span><a href="#Выводы:" data-toc-modified-id="Выводы:-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы:</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект 

Интернет-магазин  запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

## Подготовка

In [125]:
import os
import re
import nltk
import transformers
import pandas as pd 
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score, make_scorer
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.dummy import DummyClassifier

In [68]:
random_state = np.random.RandomState(454656)

In [69]:
try:
    data = pd.read_csv('datasets/toxic_comments.csv')
except:
    data = pd.read_csv('/datasets/toxic_comments.csv')

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [6]:
data.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [7]:
data.columns

Index(['Unnamed: 0', 'text', 'toxic'], dtype='object')

In [8]:
data['text'].isna().sum()

0

In [9]:
data['toxic'].isna().sum()

0

In [10]:
data = data.drop('Unnamed: 0', axis = 1)

Пропусков нет, удалили ненужный признак

In [11]:
data.sample()

,text,toxic
149742,Image\n\nThe image has been put up with permis...,0


In [72]:
len(data['text'].max())

4521

Максимальная длина текста составляет 4521 символов. Перед эмбендингом уменьшим длину строк в нужных объектах

In [74]:
data['toxic'].value_counts()

toxic
0    143106
1     16186
Name: count, dtype: int64

<b> Признаки не сбалансированы. Для чистоты обучения оставим в таком же виде.

In [75]:
data_min = data.sample(random_state=random_state, frac=0.015).reset_index(drop=True)

In [76]:
data_min.head()

,Unnamed: 0,text,toxic
0,74311,""" other than """"unlikely""""""",0
1,55750,Fish - ref desk\n\nSee your question on the re...,0
2,94942,""" (UTC)\n\n Wikipedia is not a dictionary. It...",0
3,107789,"""There you go. Thanks for spotting that! | ta...",0
4,10278,2009 (UTC)\n\nhttp://www.nachatte.com/clip/Fot...,0


In [77]:
data_min['toxic'].value_counts()

toxic
0    2149
1     240
Name: count, dtype: int64

##### Для ембендинга будем использовать модель BERT.

### BERT модель

In [16]:
tokenaizer = transformers.AutoTokenizer.from_pretrained('unitary/toxic-bert')
model = transformers.AutoModel.from_pretrained('unitary/toxic-bert')

Some weights of the model checkpoint at unitary/toxic-bert were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [81]:
futures_bert  = data_min['text'].apply(lambda x: tokenaizer.encode(x, add_special_tokens=True)[:512])

Уменьшим длину больших строк до 512 слов.

In [83]:
max_ = len(max(futures_bert, key=len))
max_

512

In [84]:
futures_bert.head(4)

0    [101, 1000, 2060, 2084, 1000, 1000, 9832, 1000...
1    [101, 3869, 1011, 25416, 4624, 2156, 2115, 316...
2    [101, 1000, 1006, 11396, 1007, 16948, 2003, 20...
3    [101, 1000, 2045, 2017, 2175, 1012, 4283, 2005...
Name: text, dtype: object

In [85]:
futures_bert  = np.array([i + [0]*(max_ - len(i)) for i in futures_bert.values])

In [86]:
futures_bert

array([[  101,  1000,  2060, ...,     0,     0,     0],
       [  101,  3869,  1011, ...,     0,     0,     0],
       [  101,  1000,  1006, ...,     0,     0,     0],
       ...,
       [  101,  2061,  1012, ...,     0,     0,     0],
       [  101,  4067,  2017, ...,     0,     0,     0],
       [  101,  2017, 24646, ...,     0,     0,     0]])

In [88]:
futures_bert_mask = np.where(np.array(futures_bert)!=0, 1,0)

In [89]:
futures_bert_mask

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])

In [90]:
futures_bert.shape

(2389, 512)

In [91]:
futures_bert_mask.shape

(2389, 512)

In [92]:
batch_size = 150
emb = []
for i in range(futures_bert.shape[0] // batch_size):
    batch = torch.LongTensor(futures_bert[batch_size*i:batch_size*(i+1)])
    attention_mask_batch = torch.LongTensor(futures_bert_mask[batch_size*i:batch_size*(i+1)])

    with torch.no_grad():
        batch_embeddings = model(batch, attention_mask=attention_mask_batch)

    emb.append(batch_embeddings[0][:,0,:].numpy()) 

In [93]:
features_bert = np.concatenate(emb)

In [94]:
features_bert.shape

(2250, 768)

In [95]:
data_min['toxic'].shape

(2389,)

In [106]:
data_min = data_min[:features_bert.shape[0]]

In [107]:
data_min['toxic'].shape

(2250,)

## Обучение 

In [108]:
make_sc = make_scorer(f1_score)

In [109]:
 data_min['toxic'].shape

(2250,)

In [110]:
features_bert.shape

(2250, 768)

Отделим тестовую выборку

In [112]:
target_train, target_test, futures_train, futures_test = train_test_split(
    data_min['toxic'], 
    features_bert, 
    test_size=0.2)

In [113]:
target_train.value_counts()

toxic
0    1620
1     180
Name: count, dtype: int64

In [114]:
target_test.value_counts()

toxic
0    404
1     46
Name: count, dtype: int64

<b> Логистическая регрессия

In [115]:
model_log_reg = LogisticRegression(random_state=random_state)

In [116]:
cross_val_score(model_log_reg, futures_train, target_train, cv=4, scoring=make_sc).mean()

C:\Users\Evnyh\Desktop\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Evnyh\Desktop\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n

0.9386649203158809

Логистическая регрессия выдает отличный результат.

<b>CatBoostClassifier

In [117]:
model_cat = CatBoostClassifier()

In [118]:
param = {
    'learning_rate': [0.8],
    'iterations': [500],
    'depth': [4,6],
    'thread_count':[-1]
}

In [120]:
model_grid = GridSearchCV(model_cat,
                         param_grid=param,
                         scoring=make_sc,
                         cv=4,
                         )

In [121]:
model_grid.fit(futures_train, target_train)

0:	learn: 0.0826467	total: 20.4ms	remaining: 10.2s
1:	learn: 0.0360649	total: 38.6ms	remaining: 9.62s
2:	learn: 0.0260556	total: 56.4ms	remaining: 9.35s
3:	learn: 0.0206517	total: 74.3ms	remaining: 9.21s
4:	learn: 0.0181479	total: 92.6ms	remaining: 9.17s
5:	learn: 0.0166499	total: 110ms	remaining: 9.07s
6:	learn: 0.0148403	total: 128ms	remaining: 9s
7:	learn: 0.0112165	total: 148ms	remaining: 9.11s
8:	learn: 0.0106663	total: 169ms	remaining: 9.21s
9:	learn: 0.0093688	total: 195ms	remaining: 9.55s
10:	learn: 0.0069370	total: 215ms	remaining: 9.57s
11:	learn: 0.0062005	total: 239ms	remaining: 9.71s
12:	learn: 0.0051359	total: 259ms	remaining: 9.7s
13:	learn: 0.0049280	total: 279ms	remaining: 9.69s
14:	learn: 0.0039725	total: 300ms	remaining: 9.69s
15:	learn: 0.0035848	total: 318ms	remaining: 9.63s
16:	learn: 0.0031011	total: 337ms	remaining: 9.57s
17:	learn: 0.0022031	total: 356ms	remaining: 9.52s
18:	learn: 0.0019740	total: 374ms	remaining: 9.46s
19:	learn: 0.0018097	total: 394ms	remain

GridSearchCV(cv=4,
             estimator=<catboost.core.CatBoostClassifier object at 0x0000025BB24A93D0>,
             param_grid={'depth': [4, 6], 'iterations': [500],
                         'learning_rate': [0.8], 'thread_count': [-1]},
             scoring=make_scorer(f1_score))

In [122]:
model_grid.best_score_

0.9289364007668585

CatBoostRegressor выдает тоже хороший результат метрики F1.

##### Для проведения теста используем логистическую регрессию.

### Тест

In [123]:
model_log_reg.fit(futures_train, target_train)

C:\Users\Evnyh\Desktop\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=RandomState(MT19937) at 0x25BB23D5740)

In [124]:
f1_score(target_test, model_log_reg.predict(futures_test))

0.9069767441860466

<b> Константная модель Dammy

In [132]:
param = {
    'strategy' : ['most_frequent', 'prior', 'stratified', 'uniform', 'constant']
}
dammy_model =  DummyClassifier()
gsv_m = GridSearchCV(dammy_model, param_grid=param, scoring='f1')
gsv_m.fit(futures_train, target_train)

C:\Users\Evnyh\Desktop\venv\Lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
5 fits failed out of a total of 25.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Evnyh\Desktop\venv\Lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Evnyh\Desktop\venv\Lib\site-packages\sklearn\dummy.py", line 196, in fit
    raise ValueError(
ValueError: Constant target value has to be specified when the constant strategy is used.

  warnings.warn(some_fits_failed_message, FitFailedWarning)
C:\Users\Evnyh\Desktop\venv\Lib\site-packag

GridSearchCV(estimator=DummyClassifier(),
             param_grid={'strategy': ['most_frequent', 'prior', 'stratified',
                                      'uniform', 'constant']},
             scoring='f1')

In [133]:
gsv_m.best_score_

0.17797197983070764

In [134]:
pred = gsv_m.predict(futures_test)
f1_score(target_test, pred)

0.12403100775193798

Константный классификатор выдает низкий показатель метрики F1,  что также подверждает актуальность нашей модели.

## Выводы:

Для выполенения поставленной задачи была использована языковая модель BERT, которая выдала хороший результат на тестовой выборке. Но для данной модели требуются большие вычеслительные мощности, желательно GPU с поддержкой CUDA и объемомо RAM не менее 12ГБ. Показатель F1 на тесовой выборке превысил минмимальный порог для текущей задачи.